In [15]:
import os
import cv2
import numpy as np
from collections import Counter

# Function to calculate IoU
def calculate_iou_class(pred_mask, gt_mask, class_id):
    pred_class = (pred_mask == class_id)
    gt_class = (gt_mask == class_id)
    intersection = np.logical_and(pred_class, gt_class)
    union = np.logical_or(pred_class, gt_class)
    iou = np.sum(intersection) / np.sum(union)
    #print(f"Accuracy of {class_id}: {iou}")
    return iou

def calculate_acc(pred_mask, gt_mask):
    acc = np.sum(pred_mask == gt_mask)
    total = pred_mask.size
    return acc / total

In [16]:
def calculate_result(pred_mask, gt_mask, labels, file_desc):
    iou_values = np.empty(len(labels))  
    if pred_mask.shape != gt_mask.shape:
        print(f"Error: Image sizes do not match")
        return

    # Calculate IoU for each class
    for ii, class_id in enumerate(labels):
        iou = calculate_iou_class(pred_mask, gt_mask, class_id)
        iou_values[ii] = iou
        acc = calculate_acc(pred_mask, gt_mask)
    return (np.mean(iou_values), file_desc, acc)

In [17]:
import os
# Path to the folder containing prediction masks and ground truth masks
prediction_folder = '../experimentation/results_output/controlnet_new/'
ground_truth = '../../../../../../mnt/data/cityscapes/gtFine/val/frankfurt/frankfurt_000000_000294_gtFine_labelIds.png'
model_prediction = '../experimentation/results_output/frankfurt_000000_000294_leftImg8bit.png'

#ground_truth = './results_check/configs/segformer/frankfurt/frankfurt_000000_000294_leftImg8bit.png'

# List all image files in the folders
prediction_files = os.listdir(prediction_folder)

# Find corresponding ground truth mask
image_mask = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE)

print(Counter(image_mask.flatten()).keys())
classes = set(Counter(image_mask.flatten()).keys())  # Find all classes to test
ignore = {0,1,2,3,4,5,6,9,10,14,15,16,18,29,30,-1}
new_classes = list(classes - ignore)
print(new_classes)


#Prompt test
x = [' Depth ' + str(i) for i in [0,0.5,1,1.5,2]]
y = [' Canny ' + str(i) for i in [0,0.5,1,1.5,2]]

#Denoising Test (Also used for model tests)
#x = ['CFG 5.0', 'CFG 7.0', 'CFG 10.0', 'CFG 12.0', 'CFG 15.0']
#y = [0.3,0.4,0.5]

#Seed Test
#x = [f"Seed {i+1}" for i in range(10)]
#y = ["CFG 5", "CFG 10"]

dict_keys([3, 2, 11, 23, 17, 4, 20, 21, 26, 24, 13, 8, 7, 1])
[7, 8, 11, 13, 17, 20, 21, 23, 24, 26]


In [18]:
import re
results = []

model_mask = cv2.imread(model_prediction, cv2.IMREAD_GRAYSCALE)
results.append(calculate_result(model_mask, model_mask, new_classes, "Model prediction"))

for i, pred_file in enumerate(prediction_files):
    pred_mask = cv2.imread(os.path.join(prediction_folder, pred_file), cv2.IMREAD_GRAYSCALE)
    results.append(calculate_result(pred_mask, model_mask, new_classes, f'{x[i % len(x)]} {y[i // len(x)]}'))
    #results.append(calculate_result(pred_mask, model_mask, new_classes, f'{pred_file}'))  
 
#results.sort()
for i in results:
    print(f"IoU: {i[0]:.4f} Acc: {i[2]:.4f} File: {i[1]}")

IoU: 1.0000 Acc: 1.0000 File: Model prediction
IoU: 0.7070 Acc: 0.9560 File:  Depth 0  Canny 0
IoU: 0.6756 Acc: 0.9611 File:  Depth 0.5  Canny 0
IoU: 0.7174 Acc: 0.9597 File:  Depth 1  Canny 0
IoU: 0.7216 Acc: 0.9622 File:  Depth 1.5  Canny 0
IoU: 0.6211 Acc: 0.9400 File:  Depth 2  Canny 0
IoU: 0.7208 Acc: 0.9633 File:  Depth 0  Canny 0.5
IoU: 0.6397 Acc: 0.9330 File:  Depth 0.5  Canny 0.5
IoU: 0.6185 Acc: 0.9240 File:  Depth 1  Canny 0.5
IoU: 0.7384 Acc: 0.9630 File:  Depth 1.5  Canny 0.5
IoU: 0.6546 Acc: 0.9576 File:  Depth 2  Canny 0.5
IoU: 0.6914 Acc: 0.9513 File:  Depth 0  Canny 1
IoU: 0.6696 Acc: 0.9414 File:  Depth 0.5  Canny 1
IoU: 0.6529 Acc: 0.9588 File:  Depth 1  Canny 1
IoU: 0.6528 Acc: 0.9596 File:  Depth 1.5  Canny 1
IoU: 0.7013 Acc: 0.9638 File:  Depth 2  Canny 1
IoU: 0.6504 Acc: 0.9579 File:  Depth 0  Canny 1.5
IoU: 0.6432 Acc: 0.9563 File:  Depth 0.5  Canny 1.5
IoU: 0.6659 Acc: 0.9454 File:  Depth 1  Canny 1.5
IoU: 0.5420 Acc: 0.9140 File:  Depth 1.5  Canny 1.5
IoU: 0.